In [1]:
spark

## Get datas from HDFS and combine into single csv using PySpark into HDFS back

In [12]:
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, StringType, FloatType
import os

# Stop any existing SparkSession
if 'spark' in locals():
    spark.stop()
    
# Create new SparkSession with minimal memory settings
spark = SparkSession.builder \
    .appName("Weather Data Combination") \
    .config("spark.jars.packages", "com.crealytics:spark-excel_2.12:0.13.7") \
    .config("spark.driver.memory", "2g") \
    .config("spark.executor.memory", "2g") \
    .config("spark.sql.shuffle.partitions", "2") \
    .config("spark.driver.maxResultSize", "1g") \
    .getOrCreate()

# Define schema with all required columns
schema = StructType([
    StructField("Line#", StringType(), True),
    StructField("Date", StringType(), True),
    StructField("Time", StringType(), True),
    StructField("Water Content (m3/m3)", FloatType(), True),
    StructField("Solar Radiation (W/m2)", FloatType(), True),
    StructField("Rain (mm)", FloatType(), True),
    StructField("Temperature (Celcius)", FloatType(), True),
    StructField("RH (%)", FloatType(), True),
    StructField("Wind Speed (m/s)", FloatType(), True),
    StructField("Gust Speed (m/s)", FloatType(), True),
    StructField("Wind Direction (Degree)", FloatType(), True),
    StructField("Dew Point (Celcius)", FloatType(), True)
])

# File paths
hdfs_directory_path = "hdfs:///user/student/"
output_path = "hdfs:///user/student/combined_raw_data.csv"

# List of files to process
file_paths = [
    f"{hdfs_directory_path}APRIL-2021.xlsx",
    f"{hdfs_directory_path}APRIL-2022.xlsx",
    f"{hdfs_directory_path}AUGUST-2021.xlsx",
    f"{hdfs_directory_path}DECEMBER-2020.xlsx",
    f"{hdfs_directory_path}DECEMBER-2021.xlsx",
    f"{hdfs_directory_path}FEBRUARY-2021.xlsx",
    f"{hdfs_directory_path}FEBRUARY-2022.xlsx",
    f"{hdfs_directory_path}JANUARY-2021.xlsx",
    f"{hdfs_directory_path}JANUARY-2022.xlsx",
    f"{hdfs_directory_path}JULY-2021.xlsx",
    f"{hdfs_directory_path}MARCH-2021.xlsx",
    f"{hdfs_directory_path}MARCH-2022.xlsx",
    f"{hdfs_directory_path}MAY-2021.xlsx",
    f"{hdfs_directory_path}NOVEMBER-2020.xlsx",
    f"{hdfs_directory_path}NOVEMBER-2021.xlsx",
    f"{hdfs_directory_path}OCTOBER-2020.xlsx",
    f"{hdfs_directory_path}OCTOBER-2021.xlsx",
    f"{hdfs_directory_path}SEPTEMBER-2020.xlsx",
    f"{hdfs_directory_path}SEPTEMBER-2021.xlsx",
    f"{hdfs_directory_path}jUNE-2021.xlsx"
]

def process_and_combine_files(file_paths, schema, output_path):
    """
    Process multiple Excel files and combine them into a single CSV file
    while ensuring all columns are present and properly typed.
    """
    combined_df = None
    
    for i, file_path in enumerate(file_paths):
        try:
            print(f"Processing file {i+1}/{len(file_paths)}: {file_path}")
            
            # Read Excel file
            current_df = spark.read.format("com.crealytics.spark.excel") \
                .option("header", "true") \
                .option("dataAddress", "'RUA Data'!A6") \
                .option("maxRowsInMemory", 1000) \
                .option("treatEmptyValuesAsNulls", "true") \
                .schema(schema) \
                .load(file_path)
            
            # Select columns in specific order to ensure consistency
            current_df = current_df.select(
                "Line#",
                "Date",
                "Time",
                "Water Content (m3/m3)",
                "Solar Radiation (W/m2)",
                "Rain (mm)",
                "Temperature (Celcius)",
                "RH (%)",
                "Wind Speed (m/s)",
                "Gust Speed (m/s)",
                "Wind Direction (Degree)",
                "Dew Point (Celcius)"
            )
            
            # Combine DataFrames
            if combined_df is None:
                combined_df = current_df
            else:
                combined_df = combined_df.union(current_df)
            
            # Clear cache after each file
            current_df.unpersist()
            spark.catalog.clearCache()
            
            print(f"Successfully processed {file_path}")
            
        except Exception as e:
            print(f"Error processing {file_path}: {str(e)}")
            continue
    
    # Write combined data to CSV
    if combined_df is not None:
        combined_df.write \
            .mode("overwrite") \
            .option("header", "true") \
            .option("compression", "none") \
            .csv(output_path)
    else:
        print("No data was successfully processed")

# Execute the combination process
try:
    process_and_combine_files(file_paths, schema, output_path)
    print("Data combination completed successfully")
except Exception as e:
    print(f"Error in main process: {str(e)}")
finally:
    # Clean up
    spark.catalog.clearCache()
    spark.stop()

Processing file 1/20: hdfs:///user/student/APRIL-2021.xlsx
Successfully processed hdfs:///user/student/APRIL-2021.xlsx
Processing file 2/20: hdfs:///user/student/APRIL-2022.xlsx
Successfully processed hdfs:///user/student/APRIL-2022.xlsx
Processing file 3/20: hdfs:///user/student/AUGUST-2021.xlsx
Successfully processed hdfs:///user/student/AUGUST-2021.xlsx
Processing file 4/20: hdfs:///user/student/DECEMBER-2020.xlsx
Successfully processed hdfs:///user/student/DECEMBER-2020.xlsx
Processing file 5/20: hdfs:///user/student/DECEMBER-2021.xlsx
Successfully processed hdfs:///user/student/DECEMBER-2021.xlsx
Processing file 6/20: hdfs:///user/student/FEBRUARY-2021.xlsx
Successfully processed hdfs:///user/student/FEBRUARY-2021.xlsx
Processing file 7/20: hdfs:///user/student/FEBRUARY-2022.xlsx
Successfully processed hdfs:///user/student/FEBRUARY-2022.xlsx
Processing file 8/20: hdfs:///user/student/JANUARY-2021.xlsx
Successfully processed hdfs:///user/student/JANUARY-2021.xlsx
Processing file 9/

Data combination completed successfully


## Read Data from HDFS using PySpark

In [ ]:
from pyspark.sql import SparkSession

# Initialize Spark session
spark = SparkSession.builder \
    .appName("Read CSV from HDFS") \
    .getOrCreate()

# Path to the CSV file on HDFS
file_path = "hdfs:///user/student/combined_raw_data.csv"

# Read the CSV file into a DataFrame
df = spark.read.csv(file_path, header=True, inferSchema=True)

# Show the DataFrame content
df.head(5)

df.printSchema()